# LangChain: Models, Prompts и Output Parsers

<img src='../images/openai_logo.jpeg' align="right" width="628" height="628" >

## Outline

 * <a href="#c1"> 🛠 Прямой запрос к API от OpenAI </a>
 * <a href="#c2"> 📂 API запрост через LangChain: :</a>
   * <a href="#look1"> 🔧 Prompts </a>
   * <a href="#check1"> 🔋 Models</a>
   * <a href="#check1"> 🤷‍ Output parsers</a>
   
* <a href="#6">🧸 Выводы и заключения</a>

## <center> Получили свой [OpenAI API Key](https://platform.openai.com/account/api-keys) или сразу используй наш

In [ ]:
from config import course_api_key
from config import openai_api_key

In [1]:
!pip install openai -q

In [25]:
import openai

openai.api_key = openai_api_key # 'Вставь сюда api_key, если он у тебя есть, либо используй наш!'

## Chat API : ключ от OpenAI или наш?!

Let's start with a direct API calls to OpenAI.

In [26]:
# import openai


 Knowledge_Base_ru.csv	 Welcome.ipynb	 config.py  'М3_Knowledge base.ipynb'
 M1_API_intro.ipynb	 __pycache__	 utils.py    Файнтюн_Llama_2.ipynb


/app/storage_local/Course/LLM_practical_course/notebooks


In [7]:
from utils import ChatCompletion

def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = ChatCompletion().create(
        course_api_key=course_api_key,
        model = model,
        messages = messages,
        temperature=0, 
        )
    
    print(response)
    return response.response.content
    

In [36]:
# from openai import ChatCompletion
# from config import openai_api_key

# def get_completion(prompt, model="gpt-3.5-turbo"):

#     messages = [{"role": "user", "content": prompt}]
    
#     response = ChatCompletion.create(
#         api_key = openai_api_key,
#         model=model,
#         messages=messages,
#         temperature=0, 
#     )
#     print(response)
#     return response.choices[0].message["content"]


In [8]:
tmp = get_completion("What is 1+1?")
tmp

ResponseSchema(response=ChatGPTEntry(role='assistant', content='1+1 equals 2.'), prompt_tokens=14, completion_tokens=7, available_tokens=498160)


'1+1 equals 2.'

## Chat API : LangChain

Let's try how we can do the same using LangChain.

<img src='../images/LangChain.webp' align="right" width="528" height="528" >

In [7]:
#!pip install --upgrade langchain

### Model

In [11]:
from langchain.chat_models import ChatOpenAI
from config import openai_api_key

# Чтобы контролировать случайность и креативность генерируемого текста 
# с помощью LLM, используйте температуру = 0,0.

# chat = ChatOpenAI(temperature=0.0, openai_api_key = openai_api_key)

In [13]:
from utils import ChatOpenAI

# Чтобы контролировать случайность и креативность генерируемого текста 
# с помощью LLM, используйте температуру = 0,0.

chat = ChatOpenAI(temperature=0.0, course_api_key = course_api_key)


### Prompt template

In [14]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [15]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [16]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [17]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [18]:
customer_style = """American English \
in a calm and respectful tone
"""

In [19]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [20]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [21]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [22]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [23]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [24]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make things even worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!


In [21]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [22]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [23]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [24]:
service_response = chat(service_messages)
# service_response = chat(service_messages)
print(service_response.content)

Arr, me hearty customer, the warranty be no match for the cost of cleanin' yer galley, for 'tis ye who be at fault fer misusin' yer blender by forgettin' to stash the lid afore ye started it. Aye, 'tis a tough break! Fare thee well, me hearties!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [25]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [26]:
print('delete_this_print')

delete_this_print


In [27]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings: \
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [28]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [29]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, course_api_key=course_api_key) 
response = chat(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}


In [30]:
type(response.content)

str

In [31]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()


In [ ]:
print(format_instructions)

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')